In [2]:
# imports
import pandas_datareader as pdr
import pandas as pd
import datetime as datetime
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
import statsmodels.api as smapi
import numpy as np
import scipy as sp
import pickle
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [25]:
class DATA():
    # economic data
    
    @classmethod
    def get_commod_data(cls,*args,**kwargs):
        data = {'oil': cls.get_oil_data, 'gasoline':cls.get_gasoline_data,
                'natgas': cls.get_natgas_data, 'corn':cls.get_corn_data,
                'wheat':cls.get_wheat_data}
        
        results = {x: data[x](*args,**kwargs) for x in data.keys()}
        return results
    
    #commodities data
    @staticmethod
    def get_oil(*args,**kwargs):
        data = quandl.get("CHRIS/CME_QM1")
        return data
    @staticmethod
    def get_gasoline(*args,**kwargs):
        data = quandl.get("CHRIS/ICE_N1")
        return data
    @staticmethod
    def get_natgas(*args, **kwargs):
        data = quandl.get("CHRIS/CME_NG2")
    @staticmethod
    def get_corn(*args, **kwargs):
        data = quandl.get("CHRIS/LIFFE_EMA2")
        return data
    @staticmethod
    def get_wheat(*args, **kwargs):
        data = quandl.get("CHRIS/CME_W4")
        return data
    # Economic data
    @staticmethod
    def get_mich_cons_sent(*args, **kwargs):
        data = quandl.get('UMICH/SOC1')
        return data
    @staticmethod
    def get_fred_gdp(*args,**kwargs):
        data = quandl.get("FRED/GDP")
        return data
    # interest rate data
    @staticmethod
    def get_us_yields(*args,**kwargs):
        data = quandl.get("USTREASURY/YIELD")
        return data

class STATS():
    @classmethod
    def AR_fit(data,*args,**kwargs):
        pass
    def ARMA_fit(data,*args,**kwargs):
        pass
    def ARIMA_fit(data,*args,**kwargs):
        pass
    def SARIMA_fit(data,*args,**kwargs):
        pass

class PLOTTERS():
    pass
